In [1]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

In [2]:
import datasets

dataset=datasets.load_dataset("paws", "labeled_final")

/home/ansh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from transformers import BertTokenizer, BertModel
# import torch
# from sklearn.metrics.pairwise import cosine_similarity

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model.to(device)

# def similarity_score(word1, word2):
#     encoded_word1 = tokenizer(word1, return_tensors='pt', padding=True, truncation=True)
#     encoded_word2 = tokenizer(word2, return_tensors='pt', padding=True, truncation=True)
#     encoded_word1.to(device)
#     encoded_word2.to(device)

#     with torch.no_grad():
#         output1 = model(**encoded_word1)
#         output2 = model(**encoded_word2)
    

#     embedding1 = output1.last_hidden_state.mean(dim=1).squeeze(0)
#     embedding2 = output2.last_hidden_state.mean(dim=1).squeeze(0)
    
#     embedding1 /= torch.norm(embedding1, dim=-1, keepdim=True)
#     embedding2 /= torch.norm(embedding2, dim=-1, keepdim=True)

#     sim_score = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))

#     return sim_score[0][0]

In [4]:
def similarity_score(word1, word2):
    if word1 not in wv or word2 not in wv:
        return 0
    return wv.similarity(word1, word2)

In [5]:
from scipy.spatial.distance import cosine, cdist
import numpy as np

def sentencesToMatrix(sentence1, sentence2):
    lis1 = sentence1.split()
    lis2 = sentence2.split()
    simMatrix=[]
    for i in lis1:
        if i not in wv:
            continue
        temp=[]
        for j in lis2:
            if j not in wv:
                continue
            temp.append(similarity_score(i,j))
        simMatrix.append(temp)
    return simMatrix

def biggestColumn(matrix):
    maxCol=[]
    for i in matrix:
        maxCol.append(max(i))
    return maxCol

## BERTSCORE
def executeBertScore(sentence1, sentence2):
    sentence1=sentence1.lower()
    sentence2=sentence2.lower()
    simMatrix = sentencesToMatrix(sentence1, sentence2)
    maxCol = biggestColumn(simMatrix)
    return sum(maxCol)/len(maxCol)
def getSentenceScore(sentence1, sentence2):
    R=executeBertScore(sentence1, sentence2)
    S=executeBertScore(sentence2, sentence1)
    return 2*(R*S)/(R+S)
    

## COSINE SIMILARITY between average vectors
def getSentenceScorec(sentence1, sentence2):
    sentence1=sentence1.lower()
    sentence2=sentence2.lower()
    sen1List = sentence1.split()
    sen2List = sentence2.split()
    emb1=[wv[word] for word in sen1List if word in wv]
    emb2=[wv[word] for word in sen2List if word in wv]
    if len(emb1)==0 or len(emb2)==0:
        return 0
    emb1 = sum(emb1)/len(emb1)
    emb2 = sum(emb2)/len(emb2)
    return 1-cosine(emb1, emb2)

##Word Movers Distance
def getSentenceScorem(sentence1, sentence2):
    sentence1=sentence1.lower()
    sentence2=sentence2.lower()
    sen1List = sentence1.split()
    sen2List = sentence2.split()
    emb1=[wv[word] for word in sen1List if word in wv]
    emb2=[wv[word] for word in sen2List if word in wv]
    distance_matrix = cdist(emb1, emb2)

    min_distances = np.min(distance_matrix, axis=1)

    total_distance = np.sum(min_distances) / len(emb1)

    return total_distance

##DEBUG
sen1="This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic ."
sen2="This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic ."

print(getSentenceScore(sen1, sen2))
print(getSentenceScorec(sen1, sen2))
print(getSentenceScorem(sen1, sen2))


0.9425530127497206
0.9792997443616231
0.23887685987189416


In [6]:
cur='test'
for i in range(10):
    print(dataset[cur][i]['sentence1'])
    print(dataset[cur][i]['sentence2'])
    print(getSentenceScore(dataset[cur][i]['sentence1'], dataset[cur][i]['sentence2']))
    print(dataset[cur][i]['label'])
    print()

This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .
0.9425530127497206
0

His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America .
His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri .
0.9853399498314827
0

In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan .
In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan inspected the venue .
0.952760600929088
1

Steiner argued that , in the right c

In [7]:
threshold=0.4
correct=0
cur='test'
for i in range(len(dataset[cur])):
    if getSentenceScore(dataset[cur][i]['sentence1'], dataset[cur][i]['sentence2'])>threshold:
        if dataset[cur][i]['label']==1:
            correct+=1
    else:
        if dataset[cur][i]['label']==0:
            correct+=1
print(correct/len(dataset[cur]))

0.442


In [8]:
import math
def createIdf(cur):
    idf={}
    for i in range(len(dataset[cur])):
        sen1=dataset[cur][i]['sentence1']
        sen2=dataset[cur][i]['sentence2']
        sen1=sen1.lower()
        sen2=sen2.lower()
        sen1List = sen1.split()
        sen2List = sen2.split()
        for word in sen1List:
            if word in wv:
                if word not in idf:
                    idf[word]=1
                else:
                    idf[word]+=1
        for word in sen2List:
            if word in wv:
                if word not in idf:
                    idf[word]=1
                else:
                    idf[word]+=1
    wordNum=0
    for word in idf:
        wordNum+=1
    for word in idf:
        idf[word]=math.log(wordNum/idf[word])
    return idf

def getSentenceScoreWithIdf(sentence1, sentence2, idf):
    sentence1=sentence1.lower()
    sentence2=sentence2.lower()
    simMatrix = sentencesToMatrix(sentence1, sentence2)
    maxCol = biggestColumn(simMatrix)
    ans=0
    curIdf=0
    for i in range(len(maxCol)):
        sen=sentence1.split()[i]
        if sen in idf:
            ans+=maxCol[i]*idf[sen] 
            curIdf+=idf[sen]
    # print(curIdf)
    if curIdf==0:
        return 0
    ans/=curIdf
    return (ans)

In [9]:
cur='test'
idf=createIdf(cur)
for i in range(20):
    print(dataset[cur][i]['sentence1'])
    print(dataset[cur][i]['sentence2'])
    print(getSentenceScoreWithIdf(dataset[cur][i]['sentence1'], dataset[cur][i]['sentence2'],idf))
    print(dataset[cur][i]['label'])
    print()

threshold=0.95
correct=0
for i in range(len(dataset[cur])):
    score=getSentenceScoreWithIdf(dataset[cur][i]['sentence1'], dataset[cur][i]['sentence2'],idf)
    # print(score)
    if score>threshold:
        if dataset[cur][i]['label']==1:
            correct+=1
    else:
        if dataset[cur][i]['label']==0:
            correct+=1
print(correct/len(dataset[cur]))

This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .
0.9349131934486553
0

His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America .
His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri .
1.0
0

In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan .
In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan inspected the venue .
0.8547474441366353
1

Steiner argued that , in the right circumstances ,